<a href="https://colab.research.google.com/github/Catatau52/Data_Mining-Data_from_CVM/blob/main/Personal_Project_Data_Mining_(CVM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Carregando as bibliotecas

In [ ]:
!pip install wget

In [2]:
import pandas as pd
import numpy as np
import wget
import decimal
import os
from zipfile import ZipFile

#Acessando a base de dados da CVM

download da base itr (trimestral) e dfp (anual) desde 2011 até 2022

In [3]:
url_base_itr = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'
url_base_dfp = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'

Criando uma lista com todos os arquivos e baixando

In [4]:
arquivos_itr_zip = []
arquivos_dfp_zip = []
for ano in range(2011,2022+1):
  arquivos_itr_zip.append(f'itr_cia_aberta_{ano}.zip')
  arquivos_dfp_zip.append(f'dfp_cia_aberta_{ano}.zip')


In [5]:
for arq in arquivos_itr_zip:
  wget.download(url_base_itr+arq)

for arq in arquivos_dfp_zip:
  wget.download(url_base_dfp+arq)

In [6]:
for arq in arquivos_itr_zip:
  ZipFile(arq, 'r').extractall('ITR')

for arq in arquivos_dfp_zip:
  ZipFile(arq, 'r').extractall('DFP')

Concatenando todos os resultados em ITR e DFP e gerando CSV

In [7]:
# Criar a pasta
os.makedirs(os.path.join('/content', 'Dados'), exist_ok=True)

docs = ['BPA_ind','BPP_ind','DRE_ind','DVA_ind']

for doc in docs:
  arquivo = pd.DataFrame()
  for ano in range(2011,2022+1):
    arquivo = pd.concat([arquivo, pd.read_csv(f'ITR/itr_cia_aberta_{doc}_{ano}.csv', sep=';', decimal=',', encoding='ISO-8859-1')])
  arquivo.to_csv(f'Dados/itr_cia_aberta_{doc}_2011-2022',index=False)

for doc in docs:
  arquivo = pd.DataFrame()
  for ano in range(2011,2022+1):
    arquivo = pd.concat([arquivo, pd.read_csv(f'DFP/dfp_cia_aberta_{doc}_{ano}.csv', sep=';', decimal=',', encoding='ISO-8859-1')])
  arquivo.to_csv(f'Dados/dfp_cia_aberta_{doc}_2011-2022',index=False)

Carregando data frames

In [735]:
BPA_itr = pd.read_csv(f'/content/Dados/itr_cia_aberta_BPA_ind_2011-2022', sep=',', decimal=',')
BPP_itr = pd.read_csv(f'/content/Dados/itr_cia_aberta_BPP_ind_2011-2022', sep=',', decimal=',')
DVA_itr = pd.read_csv(f'/content/Dados/itr_cia_aberta_DVA_ind_2011-2022', sep=',', decimal=',')
DRE_itr = pd.read_csv(f'/content/Dados/itr_cia_aberta_DRE_ind_2011-2022', sep=',', decimal=',')

BPA_dfp = pd.read_csv(f'/content/Dados/dfp_cia_aberta_BPA_ind_2011-2022', sep=',', decimal=',')
BPP_dfp = pd.read_csv(f'/content/Dados/dfp_cia_aberta_BPP_ind_2011-2022', sep=',', decimal=',')
DVA_dfp = pd.read_csv(f'/content/Dados/dfp_cia_aberta_DVA_ind_2011-2022', sep=',', decimal=',')
DRE_dfp = pd.read_csv(f'/content/Dados/dfp_cia_aberta_DRE_ind_2011-2022', sep=',', decimal=',')

#Reescrevendo empresas que mudaram de nome ao longo dos anos.

In [736]:
dataframes = [BPA_itr, BPP_itr, DVA_itr, DRE_itr, BPA_dfp, BPP_dfp, DVA_dfp, DRE_dfp]

for df in dataframes:
    df['DENOM_CIA'] = df['DENOM_CIA'].str.replace('SENIOR SOLUTION S.A.', 'SINQIA S.A.')

<ipython-input-736-caace5e2e005>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['DENOM_CIA'] = df['DENOM_CIA'].str.replace('SENIOR SOLUTION S.A.', 'SINQIA S.A.')


#Filtrando dados

filtrandos os dados trimestrais relevantes para BPA

In [737]:
BPA_itr = BPA_itr.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)
#BPA_itr = BPA_itr[BPA_itr['CD_CONTA'].str.len() <= 7]

#reodena colunas
BPA_itr = BPA_itr[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
#valores_a_manter = ['Ativo Total','Ativo Circulante','Disponibilidades','Caixa e Equivalentes de Caixa']
valores_a_manter = ['1','1.01','1.01.01']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPA_itr = BPA_itr.loc[BPA_itr['CD_CONTA'].isin(valores_a_manter)]

#separando os resultados
BPA_itr = BPA_itr[BPA_itr['ORDEM_EXERC'] == 'ÚLTIMO']
BPA_itr = BPA_itr.drop(['ORDEM_EXERC'],axis=1)

In [738]:
BPA_itr['CNPJ_CIA'].value_counts()

02.291.077/0001-93    111
00.192.769/0001-59    111
16.590.234/0001-76    111
15.101.405/0001-93    111
73.903.718/0001-61    111
                     ... 
08.887.330/0001-52      3
09.347.916/0001-97      3
03.468.802/0001-19      3
51.468.056/0001-06      3
60.394.723/0001-44      3
Name: CNPJ_CIA, Length: 1121, dtype: int64

In [739]:
BPA_dfp = BPA_dfp.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)
#BPA_itr = BPA_itr[BPA_itr['CD_CONTA'].str.len() <= 7]

#reodena colunas
BPA_dfp = BPA_dfp[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
#valores_a_manter = ['Ativo Total','Ativo Circulante','Disponibilidades','Caixa e Equivalentes de Caixa']
valores_a_manter = ['1','1.01','1.01.01']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPA_dfp = BPA_dfp.loc[BPA_dfp['CD_CONTA'].isin(valores_a_manter)]

#separando os resultados
BPA_dfp = BPA_dfp[BPA_dfp['ORDEM_EXERC'] == 'ÚLTIMO']
BPA_dfp = BPA_dfp.drop(['ORDEM_EXERC'],axis=1)

In [740]:
BPA_dfp['CNPJ_CIA'].value_counts()

50.746.577/0001-15    39
61.082.004/0001-50    39
00.000.000/0001-91    36
17.281.106/0001-03    36
29.950.060/0001-57    36
                      ..
12.499.629/0001-07     3
51.468.056/0001-06     3
12.592.746/0001-10     3
12.668.780/0001-21     3
81.616.807/0001-55     3
Name: CNPJ_CIA, Length: 1092, dtype: int64

filtrandos os dados trimestrais relevantes para BPP

In [741]:
BPP_itr = BPP_itr.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
BPP_itr = BPP_itr[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
#valores_a_manter = ['Passivo Total','Passivo Circulante'']
valores_a_manter = ['2','2.01','2.02'] #,'2.05','2.07'
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPP_itr_1 = BPP_itr.loc[BPP_itr['CD_CONTA'].isin(valores_a_manter)]

#valores_a_manter = ['Patrimônio Líquido']
valores_a_manter = ['Patrimônio Líquido']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPP_itr_2 = BPP_itr.loc[BPP_itr['DS_CONTA'].isin(valores_a_manter)]


BPP_itr = pd.concat([BPP_itr_1,BPP_itr_2]).sort_values(by=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM','DT_REFER','CD_CONTA'])


#separando os resultados
BPP_itr = BPP_itr[BPP_itr['ORDEM_EXERC'] == 'ÚLTIMO']
BPP_itr = BPP_itr.drop(['ORDEM_EXERC'],axis=1)

In [742]:
BPP_itr['CNPJ_CIA'].value_counts()

00.192.769/0001-59    148
02.397.080/0001-96    148
16.590.234/0001-76    148
15.101.405/0001-93    148
02.291.077/0001-93    148
                     ... 
33.066.408/0001-15      4
67.915.785/0001-01      4
09.123.654/0001-87      4
09.347.916/0001-97      4
26.643.756/0001-42      4
Name: CNPJ_CIA, Length: 1121, dtype: int64

In [743]:
BPP_dfp = BPP_dfp.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
BPP_dfp = BPP_dfp[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
#valores_a_manter = ['Passivo Total','Passivo Circulante'']
valores_a_manter = ['2','2.01','2.02'] #,'2.05','2.07'
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPP_dfp_1 = BPP_dfp.loc[BPP_dfp['CD_CONTA'].isin(valores_a_manter)]

#valores_a_manter = ['Patrimônio Líquido']
valores_a_manter = ['Patrimônio Líquido']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPP_dfp_2 = BPP_dfp.loc[BPP_dfp['DS_CONTA'].isin(valores_a_manter)]


BPP_dfp = pd.concat([BPP_dfp_1,BPP_dfp_2]).sort_values(by=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM','DT_REFER','CD_CONTA'])

#separando os resultados
BPP_dfp = BPP_dfp[BPP_dfp['ORDEM_EXERC'] == 'ÚLTIMO']
BPP_dfp = BPP_dfp.drop(['ORDEM_EXERC'],axis=1)

In [744]:
BPP_dfp['CNPJ_CIA'].value_counts()

61.082.004/0001-50    52
50.746.577/0001-15    52
00.000.000/0001-91    48
16.404.287/0001-55    48
16.590.234/0001-76    48
                      ..
84.429.752/0001-62     4
12.668.792/0001-56     4
12.668.780/0001-21     4
01.355.994/0001-21     4
03.076.274/0001-52     4
Name: CNPJ_CIA, Length: 1092, dtype: int64

filtrandos os dados trimestrais relevantes para DVA

In [745]:
DVA_itr = DVA_itr.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
DVA_itr = DVA_itr[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['Juros sobre o Capital Próprio','Dividendos']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
DVA_itr = DVA_itr.loc[DVA_itr['DS_CONTA'].isin(valores_a_manter)]

#separando os resultados
DVA_itr = DVA_itr[DVA_itr['ORDEM_EXERC'] == 'ÚLTIMO']
DVA_itr = DVA_itr.drop(['ORDEM_EXERC'],axis=1)

In [746]:
DVA_itr['CNPJ_CIA'].value_counts()

00.336.701/0001-04    84
01.104.937/0001-70    77
45.987.245/0001-92    76
00.359.742/0001-08    76
02.291.077/0001-93    74
                      ..
67.915.785/0001-01     2
09.123.654/0001-87     2
09.347.916/0001-97     2
11.306.912/0001-02     2
13.552.070/0001-02     2
Name: CNPJ_CIA, Length: 1121, dtype: int64

In [747]:
DVA_dfp = DVA_dfp.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
DVA_dfp = DVA_dfp[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['Juros sobre o Capital Próprio','Dividendos']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
DVA_dfp = DVA_dfp.loc[DVA_dfp['DS_CONTA'].isin(valores_a_manter)]

#separando os resultados
DVA_dfp = DVA_dfp[DVA_dfp['ORDEM_EXERC'] == 'ÚLTIMO']
DVA_dfp = DVA_dfp.drop(['ORDEM_EXERC'],axis=1)

In [748]:
DVA_dfp['CNPJ_CIA'].value_counts()

01.104.937/0001-70    36
00.359.742/0001-08    30
59.789.545/0001-71    28
71.476.527/0001-35    27
61.082.004/0001-50    26
                      ..
12.395.010/0001-52     2
08.572.741/0001-59     2
03.076.274/0001-52     2
03.392.498/0001-73     2
81.616.807/0001-55     2
Name: CNPJ_CIA, Length: 1090, dtype: int64


filtrandos os dados trimestrais relevantes para DRE

In [749]:
DRE_itr = DRE_itr.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
DRE_itr = DRE_itr[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_INI_EXERC','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['3.01','3.03','3.05','3.07','3.11','3.13','3.99']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
DRE_itr = DRE_itr.loc[DRE_itr['CD_CONTA'].isin(valores_a_manter)]

#separando os resultados
DRE_itr = DRE_itr[DRE_itr['ORDEM_EXERC'] == 'ÚLTIMO']
DRE_itr = DRE_itr.drop(['ORDEM_EXERC'],axis=1)

In [750]:
DRE_itr['CNPJ_CIA'].value_counts()

30.306.294/0001-45    427
15.144.017/0001-90    420
65.654.303/0001-73    420
62.144.175/0001-20    420
60.872.504/0001-23    420
                     ... 
26.643.756/0001-42      6
13.552.070/0001-02      6
03.468.802/0001-19      6
09.123.654/0001-87      6
60.394.723/0001-44      6
Name: CNPJ_CIA, Length: 1121, dtype: int64

In [751]:
DRE_dfp = DRE_dfp.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
DRE_dfp = DRE_dfp[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_INI_EXERC','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['3.01','3.03','3.05','3.07','3.11','3.13','3.99']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
DRE_dfp = DRE_dfp.loc[DRE_dfp['CD_CONTA'].isin(valores_a_manter)]

#separando os resultados
DRE_dfp = DRE_dfp[DRE_dfp['ORDEM_EXERC'] == 'ÚLTIMO']
DRE_dfp = DRE_dfp.drop(['ORDEM_EXERC','DT_INI_EXERC'],axis=1)

In [752]:
DRE_dfp['CNPJ_CIA'].value_counts()

14.388.334/0001-99    84
15.144.017/0001-90    84
60.872.504/0001-23    84
62.144.175/0001-20    84
02.682.287/0001-02    84
                      ..
08.572.741/0001-59     6
13.608.583/0001-80     6
12.395.010/0001-52     6
08.253.728/0001-37     6
81.616.807/0001-55     6
Name: CNPJ_CIA, Length: 1091, dtype: int64

Padronizando a DRE ITR para resultados acumulados

In [753]:
# Converter as colunas datas para o tipo datetime
DRE_itr['DT_INI_EXERC'] = pd.to_datetime(DRE_itr['DT_INI_EXERC'])
DRE_itr['DT_FIM_EXERC'] = pd.to_datetime(DRE_itr['DT_FIM_EXERC'])
DRE_itr['DT_REFER'] = pd.to_datetime(DRE_itr['DT_REFER'])
# Calcular a diferença de tempo entre DT_INI_EXERC e DT_FIM_EXERC
DRE_itr['PERIODO'] = DRE_itr['DT_FIM_EXERC'] - DRE_itr['DT_INI_EXERC']
#Reogarinzando a base
DRE_itr = DRE_itr.sort_values(['CNPJ_CIA','DENOM_CIA','CD_CVM','DT_REFER'])
DRE_itr = DRE_itr.reset_index(drop=True)
# Calcular a diferença de um resultado para o outro
DRE_itr['PERIODO_DIF'] = DRE_itr.groupby(['CNPJ_CIA','DT_REFER'])['PERIODO'].diff()
# Substituindo 'NaT' por '0 days'
DRE_itr['PERIODO_DIF'] = DRE_itr['PERIODO_DIF'].fillna(pd.Timedelta(0))
# Eliminar as linhas de resultados trimestrais e manter os acumulados (como nos demais documentos)
DRE_itr = DRE_itr[DRE_itr['PERIODO_DIF'] >= '-5 days' ]
DRE_itr = DRE_itr.drop(['PERIODO_DIF','PERIODO'], axis=1)
DRE_itr = DRE_itr.drop('DT_INI_EXERC', axis = 1)

In [754]:
#DRE_itr[(DRE_itr['CNPJ_CIA'] == '32.372.085/0001-25') & (DRE_itr['DT_REFER'] >= '2019-03-31')].head(10)

In [755]:
#DRE_itr[(DRE_itr['PERIODO_DIF'] < '-6 days') & (DRE_itr['PERIODO_DIF'] >= '-31 days')].head(60)

#Concatenando as bases de dados

In [756]:
df1 = pd.concat([BPA_itr,BPP_itr]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])
df2 = pd.concat([DRE_itr,DVA_itr]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])
df_itr = pd.concat([df1,df2]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])

df_itr['DT_FIM_EXERC']= pd.to_datetime(df_itr['DT_FIM_EXERC'])
df_itr['DT_REFER']= pd.to_datetime(df_itr['DT_REFER'])

df_itr = df_itr.sort_values(['CNPJ_CIA','DT_REFER'])

<ipython-input-756-03f810631ad7>:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df2 = pd.concat([DRE_itr,DVA_itr]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])


In [757]:
df3 = pd.concat([BPA_dfp,BPP_dfp]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])
df4 = pd.concat([DRE_dfp,DVA_dfp]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])
df_dfp = pd.concat([df3,df4]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])

df_dfp['DT_FIM_EXERC']= pd.to_datetime(df_dfp['DT_FIM_EXERC'])
df_dfp['DT_REFER']= pd.to_datetime(df_dfp['DT_REFER'])

df_dfp = df_dfp.sort_values(['CNPJ_CIA','DT_REFER'])

In [758]:
df = pd.concat([df_itr,df_dfp]).sort_values(by=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM','DT_REFER','CD_CONTA'])
df = df.reset_index(drop=True)
df = df.drop_duplicates()

mesclando, padronizando e removendo nomes irrelevantes do balanço

In [760]:
df['DS_CONTA'].value_counts()

Patrimônio Líquido                                                    31416
Passivo Total                                                         31413
Dividendos                                                            31398
Ativo Total                                                           31381
Juros sobre o Capital Próprio                                         31339
Passivo Circulante                                                    31229
Ativo Circulante                                                      31216
Lucro/Prejuízo do Período                                             31191
Caixa e Equivalentes de Caixa                                         29685
Passivo Não Circulante                                                29519
Resultado Antes do Resultado Financeiro e dos Tributos                29481
Resultado Bruto                                                       29479
Resultado Antes dos Tributos sobre o Lucro                            29360
Lucro por Aç

In [761]:
#df.loc[df['DS_CONTA'] == 'Outras Receitas e Despesas Operacionais']

In [762]:
df['DS_CONTA'] = df['DS_CONTA'].replace({

    'Passivos Financeiros ao Custo Amortizado' : 'Passivo Circulante',

    'Caixa e equivalentes de caixa': 'Caixa', #LOJAS HERING S.A.
    'Disponibilidades': 'Caixa',

    'Lucro por Ação - (Reais / Ação)': 'Lucro por Ação',
    'Lucro por Ação - (R$ / Ação)': 'Lucro por Ação',
    'Lucro por Ação (R$/Ação)' : 'Lucro por Ação',

    'Lucro ou Prejuízo Líquido do Período' : 'Lucro/Prejuízo do Período',

    'a': 'Receitas',
    'D': 'Receitas',
    'Receita Bruta de Venda de Bens e/ou Serviços' : 'Receitas',
    'Receita de Venda de Bens e/ou Serviços' : 'Receitas',
    'Receita' : 'Receitas',
    'Receitas das Operações' : 'Receitas',
    'Receitas da Intermediação Financeira' : 'Receitas',
    'Receitas de Intermediação Financeira' : 'Receitas',

    'Resultado Antes Tributação/Participações' : 'Resultado Antes dos Tributos sobre o Lucro',
    'Lucro ou Prejuízo das Operações Continuadas' : 'Resultado Antes dos Tributos sobre o Lucro',
    'Resultado Líquido das Operações Continuadas' : 'Resultado Antes dos Tributos sobre o Lucro',

    'Resultado Operacional' : 'Resultado Antes do Resultado Financeiro e dos Tributos',
    'Resultado antes dos Tributos sobre o Lucro' : 'Resultado Antes do Resultado Financeiro e dos Tributos',

    'Resultado Bruto Intermediação Financeira' : 'Resultado Bruto',
    'Resultado Bruto de Intermediação Financeira' : 'Resultado Bruto',

    '0': 'Ativo Total',
    'A': 'Ativo Total',
    'Ativo total:': 'Ativo Total',
    'Ativo': 'Ativo Total',
})

In [763]:
df = df[~df['DS_CONTA'].isin(['Reversão dos Juros sobre Capital Próprio','Passivos Financeiros Avaliados ao Valor Justo através do Resultado','Outros Passivos',
                              'Passivo Exigível a Longo Prazo','Outras Receitas e Despesas Operacionais'])]

In [764]:
# Substituir o conteúdo da coluna 'DS_CONTA' para 'caixa' onde 'CD_CONTA' for igual a '1.01'
df.loc[df['CD_CONTA'] == '1.01', 'DS_CONTA'] = 'Ativo Circulante'
df.loc[df['CD_CONTA'] == '1.01.01', 'DS_CONTA'] = 'Caixa'

In [765]:
df = df.loc[df['DS_CONTA'] != 'Passivo Não Circulante']

Padronizandos os nomes dos balanços

In [766]:
df['DS_CONTA'] = df['DS_CONTA'].replace({
    'Resultado Antes do Resultado Financeiro e dos Tributos': 'EBITDA',
    'Resultado Antes dos Tributos sobre o Lucro': 'EBIT',
    'Juros sobre o Capital Próprio': 'JCP',
    'Lucro/Prejuízo do Período': 'Lucro Líquido',
    'Resultado Bruto': 'Lucro Bruto',
    'Lucro/Prejuízo do Período': 'Lucro',
    'Receitas': 'Receita Total'

})

In [767]:
df['DS_CONTA'].value_counts()

Patrimônio Líquido    31416
Passivo Circulante    31414
Passivo Total         31413
Ativo Total           31401
Ativo Circulante      31401
Caixa                 31401
Dividendos            31398
EBITDA                31376
EBIT                  31376
Lucro                 31376
Lucro Bruto           31374
Receita Total         31373
Lucro por Ação        31371
JCP                   31339
Name: DS_CONTA, dtype: int64

#Transpor a coluna DS_CONTA para colunas individuais

eliminar colunas que não serão mais necessárias, mas para não perder informação da coluna "ESCALA_MOEDA" é necessário mulitplicar ou divididir a coluna "VL_CONTA"

In [768]:
df['VL_CONTA'] = df['VL_CONTA'].astype('float64')
pd.set_option('display.float_format', '{:.4f}'.format)
# Mapeamento para fatores multiplicativos
escala_mapping = {'UNIDADE': 1000}

# Função personalizada para multiplicar por fator de escala
def multiply_by_scale(row):
    escala = row['ESCALA_MOEDA']
    if escala in escala_mapping:
        return row['VL_CONTA'] / escala_mapping[escala]
    else:
        return row['VL_CONTA']

# Aplicar a função personalizada na coluna 'VL_CONTA'
df['VL_CONTA'] = df.apply(multiply_by_scale, axis=1)

df = df.drop(['DT_FIM_EXERC','CD_CONTA','ESCALA_MOEDA'], axis =1)

In [769]:
df = df.pivot_table(index=['CNPJ_CIA','DENOM_CIA','CD_CVM','DT_REFER'], columns='DS_CONTA', values='VL_CONTA' ).reset_index()
#reodena colunas
df = df[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER', 'Patrimônio Líquido', 'Ativo Total', 'Ativo Circulante', 'Passivo Total','Passivo Circulante', 'Receita Total',
         'Lucro Bruto','EBIT', 'EBITDA', 'Lucro', 'Caixa', 'Dividendos', 'JCP', 'Lucro por Ação']]
df = df.rename_axis(None, axis='columns')

In [770]:
#df.loc[df['DENOM_CIA'] == 'NATURA &CO HOLDING S.A.']

#Filtrando a base de dados para empresas relevantes
**É possível implementar mais filtros**

In [771]:
df['CNPJ_CIA'].value_counts()

15.101.405/0001-93    49
00.000.000/0001-91    48
17.155.730/0001-64    48
15.413.826/0001-50    48
16.404.287/0001-55    48
                      ..
33.937.681/0001-78     1
81.616.807/0001-55     1
03.468.802/0001-19     1
10.265.949/0001-77     1
60.394.723/0001-44     1
Name: CNPJ_CIA, Length: 1131, dtype: int64

removendo empresas que não apresentam os resultados de no mínimo 4 trismestres (59 removidas)

In [772]:
contagem = df['CNPJ_CIA'].value_counts()
df = df[df['CNPJ_CIA'].isin(contagem[contagem >= 4].index)]
# Iterar sobre os resultados da contagem
#for valor, count in contagem.items():
    #print(f"Valor: {valor}, Contagem: {count}")

In [773]:
#df.loc[df['CNPJ_CIA'] == '49.263.189/0001-02']

Removendo empresas com menos de 10milhões em Ativo Total (226 removidas)

In [774]:
df_filtrado = df[df['Ativo Total'] < 10000]
contagem = df_filtrado['CNPJ_CIA'].value_counts()
#Filtro as empresas que aparecem mais que 3x vezes para evitar balanços incosistentes
df_filtrado = df_filtrado[df_filtrado['CNPJ_CIA'].isin(contagem[contagem >= 4].index)]
#for valor, count in contagem.items():
#    print(f"Valor: {valor}, Contagem: {count}")
df = df[~df['CNPJ_CIA'].isin(df_filtrado['CNPJ_CIA'])]


Removendo empresas com menos de 100mil de Receita Total (275 removidas)

In [775]:
#df_filtrado = df[df['Receita Total'] < 100]
#contagem = df_filtrado['CNPJ_CIA'].value_counts()
#Filtro as empresas que aparecem mais que 3x vezes para evitar balanços incosistentes
#df_filtrado = df_filtrado[df_filtrado['CNPJ_CIA'].isin(contagem[contagem >= 4].index)]
#for valor, count in contagem.items():
#    print(f"Valor: {valor}, Contagem: {count}")
#df = df[~df['CNPJ_CIA'].isin(df_filtrado['CNPJ_CIA'])]


preenchendo valores com 0 para Dy, JCP e LPA.

In [776]:
df[['Dividendos','JCP','Lucro por Ação']] = df[['Dividendos','JCP','Lucro por Ação']].fillna(0)

#Inserindo Indicadores em um novo DF

In [777]:
# Vetor de indicadores
indicadores = ['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','VPA','LPA','PAYOUT','PL/ATIVOS',
                'PS/ATIVOS','DIV/PL','DIV/EBIT','DIV/EBITDA','M.EBITDA','ROE','ROIC','ROA','ILC','MLB','MLO','MLL','VLE']
# Criar novo DataFrame com coluna de indicadores repetidos
df2 = pd.DataFrame(columns=[indicadores])
df2 = df2.assign(**{coluna: df[coluna] for coluna in ['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER']})

Calculando os indicadores a partir dos balanços das empresas

In [778]:
df2['LPA'] = df['Lucro por Ação']
df2['VPA'] = df['Patrimônio Líquido'] / ( df['Lucro'] / df['Lucro por Ação'] )
df2['PAYOUT'] = ( df['Dividendos'] + df['JCP'] ) / ( df['Lucro'])

df2['PL/ATIVOS'] = df['Patrimônio Líquido'] / df['Ativo Total']
df2['PS/ATIVOS'] = df['Passivo Total'] / df['Ativo Total']
df2['DIV/PL'] = (df['Passivo Circulante'] - df['Caixa']) / df['Patrimônio Líquido']
df2['DIV/EBIT'] = (df['Passivo Circulante'] - df['Caixa']) / df['EBIT']
df2['DIV/EBITDA'] = (df['Passivo Circulante'] - df['Caixa']) / df['EBITDA']

df2['M.EBITDA'] = df['EBITDA'] / df['Receita Total']
df2['ROE'] = df['Lucro'] / df['Patrimônio Líquido']
df2['ROA'] = df['Lucro'] / df['Ativo Total']
df2['ROIC'] = df['EBIT'] / (df['Patrimônio Líquido'] + (df['Passivo Total'] - df['Passivo Circulante']))

df2['ILC'] = df['Ativo Circulante'] / df['Passivo Circulante']
df2['MLB'] = df['Lucro Bruto'] / df['Receita Total']
df2['MLO'] = df['EBIT'] / df['Receita Total']
df2['MLL'] = df['Lucro'] / df['Receita Total']

# Calcular a variação percentual do lucro
df2['VLE'] = df.groupby('CNPJ_CIA')['Lucro'].pct_change()
#df2['VLE'] = (df['Lucro'] / df['Lucro'].shift(1)) - 1

In [779]:
df2.head(60)

,CNPJ_CIA,DENOM_CIA,CD_CVM,DT_REFER,VPA,LPA,PAYOUT,PL/ATIVOS,PS/ATIVOS,DIV/PL,...,DIV/EBITDA,M.EBITDA,ROE,ROIC,ROA,ILC,MLB,MLO,MLL,VLE
0,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-03-31,0.0000,0.0000,0.4000,0.0662,1.0000,10.0505,...,118.6082,0.2191,0.0562,0.0146,0.0037,0.8585,0.3157,0.2197,0.1453,NaN
1,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-06-30,18.9728,2.1891,0.4000,0.0665,1.0000,9.8713,...,56.8123,0.2240,0.1154,0.0302,0.0077,0.8653,0.3207,0.2266,0.1488,1.1356
2,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-09-30,19.6745,3.2000,0.4000,0.0655,1.0000,9.7980,...,43.9160,0.1860,0.1626,0.0368,0.0107,0.8965,0.2678,0.1880,0.1356,0.4617
3,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-12-31,20.3217,4.2802,0.4000,0.0653,1.0000,9.6865,...,32.6106,0.1917,0.2106,0.0464,0.0138,0.9061,0.2732,0.1937,0.1360,0.3380
4,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-03-31,20.8758,0.8915,0.4000,0.0655,1.0000,9.4716,...,164.7938,0.1528,0.0427,0.0088,0.0028,0.9389,0.2823,0.1539,0.1135,-0.7914
5,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-06-30,21.6370,1.9390,0.4000,0.0644,1.0000,9.4589,...,77.7476,0.1595,0.0896,0.0180,0.0058,0.9202,0.2576,0.1609,0.1175,1.1750
6,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-09-30,22.2892,2.9188,0.4000,0.0632,1.0000,9.8968,...,58.3499,0.1561,0.1310,0.0256,0.0083,0.9017,0.2658,0.1580,0.1205,0.5048
7,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-12-31,22.9888,4.3023,0.4000,0.0627,1.0000,10.0228,...,42.2973,0.1718,0.1871,0.0378,0.0117,0.8738,0.2771,0.1854,0.1357,0.4724
8,00.000.000/0001-91,BCO BRASIL S.A.,1023,2013-03-31,21.6862,0.9057,0.4000,0.0559,1.0000,11.2306,...,199.4741,0.1650,0.0418,0.0076,0.0023,0.8781,0.3213,0.1654,0.1224,-0.7902
9,00.000.000/0001-91,BCO BRASIL S.A.,1023,2013-06-30,21.9443,3.5212,0.4000,0.0549,1.0000,11.6000,...,145.6190,0.1065,0.1605,0.0321,0.0088,0.8772,0.2486,0.3175,0.2145,2.8843


Essa situação está ocorrendo por que essas empresas não reportaram um ou mais balanços no período. (as empresas são obrigadas a reportarem anualmente apenas)

In [780]:
df[df.isna().any(axis=1)]

,CNPJ_CIA,DENOM_CIA,CD_CVM,DT_REFER,Patrimônio Líquido,Ativo Total,Ativo Circulante,Passivo Total,Passivo Circulante,Receita Total,Lucro Bruto,EBIT,EBITDA,Lucro,Caixa,Dividendos,JCP,Lucro por Ação
21069,31.001.230/0001-07,CELEO REDES TRANSMISSÃO DE ENERGIA S.A.,24597,2018-06-30,0.0000,NaN,NaN,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000
21210,32.372.085/0001-25,COBUCCIO SECURITIZADORA DE CRÉDITOS S.A.,24724,2018-12-31,1.0000,1.0000,1.0000,1.0000,0.0000,NaN,NaN,NaN,NaN,NaN,1.0000,0.0000,0.0000,0.0000
21211,32.372.085/0001-25,COBUCCIO SECURITIZADORA DE CRÉDITOS S.A.,24724,2019-03-31,1.0000,1.0000,1.0000,1.0000,0.0000,NaN,NaN,NaN,NaN,NaN,1.0000,0.0000,0.0000,0.0000
21284,32.785.497/0001-97,NATURA &CO HOLDING S.A.,24783,2019-06-30,0.1000,0.1000,0.1000,0.1000,0.0000,NaN,NaN,NaN,NaN,NaN,0.1000,0.0000,0.0000,0.0000


In [781]:
df.head(60)

,CNPJ_CIA,DENOM_CIA,CD_CVM,DT_REFER,Patrimônio Líquido,Ativo Total,Ativo Circulante,Passivo Total,Passivo Circulante,Receita Total,Lucro Bruto,EBIT,EBITDA,Lucro,Caixa,Dividendos,JCP,Lucro por Ação
0,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-03-31,52172709.0000,787908489.0000,460670797.0000,787908489.0000,536607202.0000,20179781.0000,6371709.0000,4432807.0000,4420967.0000,2932363.0000,12244142.0000,449024.0000,723921.0000,0.0000
1,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-06-30,54276101.0000,816131152.0000,479955357.0000,816131152.0000,554643276.0000,42097733.0000,13502837.0000,9539565.0000,9430620.0000,6262367.0000,18868357.0000,1044346.0000,1460601.0000,2.1891
2,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-09-30,56279126.0000,859280120.0000,512077755.0000,859280120.0000,571192875.0000,67494104.0000,18072581.0000,12687580.0000,12556373.0000,9153653.0000,19767405.0000,1405060.0000,2256401.0000,3.2000
3,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-12-31,58148690.0000,890352257.0000,518716710.0000,890352257.0000,572481618.0000,90080002.0000,24614257.0000,17448327.0000,17272140.0000,12247330.0000,9227217.0000,1847625.0000,3051308.0000,4.2802
4,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-03-31,59817963.0000,913866693.0000,545372613.0000,913866693.0000,580836511.0000,22500822.0000,6351239.0000,3463122.0000,3438060.0000,2554434.0000,14265629.0000,181408.0000,840365.0000,0.8915
5,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-06-30,61998776.0000,962760903.0000,554627497.0000,962760903.0000,602726998.0000,47296102.0000,12183060.0000,7611554.0000,7542866.0000,5555940.0000,16287118.0000,531682.0000,1690693.0000,1.9390
6,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-09-30,63844116.0000,1009923655.0000,582005138.0000,1009923655.0000,645454593.0000,69355842.0000,18433394.0000,10958083.0000,10828662.0000,8360467.0000,13603652.0000,835926.0000,2508259.0000,2.9188
7,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-12-31,65776796.0000,1049283624.0000,585819632.0000,1049283624.0000,670456746.0000,90745179.0000,25143726.0000,16824075.0000,15586534.0000,12309870.0000,11189103.0000,1570156.0000,3353792.0000,4.3023
8,00.000.000/0001-91,BCO BRASIL S.A.,1023,2013-03-31,61848822.0000,1105744898.0000,620472390.0000,1105744898.0000,706574074.0000,21106633.0000,6780966.0000,3492006.0000,3482154.0000,2583016.0000,11974401.0000,279429.0000,753777.0000,0.9057
9,00.000.000/0001-91,BCO BRASIL S.A.,1023,2013-06-30,62527717.0000,1138813786.0000,647797545.0000,1138813786.0000,738514308.0000,46784697.0000,11628837.0000,14854820.0000,4980933.0000,10033321.0000,13195754.0000,2457310.0000,1556018.0000,3.5212


In [782]:
#df_teste.to_csv('df_teste.csv', index=True, header = True)

In [783]:
df.to_csv('balanço_empresas_2011-2022.csv', index=True, header = True)
df2.to_csv('indicadores_empresas_2011-2022.csv', index=True, header = True)

#Considerações

O tratamento para a mineração adequada da base de dados foi realizado de forma meticulosa, abordando todas as possibilidades. Durante o desenvolvimento, foram identificadas diversas inconsistências na mineração de dados, mas mesmo assim, foi possível alcançar o principal objetivo: garantir que não houvesse nenhum dado "NaN" de empresas relevantes na base (apenas três empresa apresentou "NaN"), resultante de problemas na divulgação de resultados. Vale ressaltar que, apesar disso, ainda é necessário realizar o Feature Engineering para aprimorar a base de dados.

Além disso, quanto maior e mais sólida a empresa, melhor é a qualidade dos dados. Para garantir a integridade da base, foi implementado um filtro inicial para excluir empresas muito pequenas. É importante destacar que nem todas as empresas com capital aberto estão listadas em bolsa. Após a aplicação do filtro, restaram 846 empresas, das 1131 inicialmente consideradas, no período entre 2011 e 2022. Mais filtros podem ser aplicados.

Por fim, os indicadores foram calculados para uma ideia inicial, porém, seria interessante deixá-los para depois, sendo assim, a base de dados está pronta para etapa seguinte : **Tratamentos dos dados ou Featuring Engineer.**